# Neural Network Model
In this notebook, we built a neural network as a recommender system on the Goodreads dataset.

Input: 1.user indexes 2. book indexes
Layers:

1. Embedding Layer
    - maps each user index to a "u" vector in R^k 
    - maps each book index to a "v" vector in R^k 
2. Merge 
    - multiplies u and v element wise
3. Flatten
4. Dense Layer: one node
5. Activation: "Relu"
6. Output: one node

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.linalg as la
import gc

import numpy as np

import keras
from keras.models import Model
from keras.layers import merge
from keras.layers import Input, Dense, Dropout, Embedding, Flatten
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10862299546963507758
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13068468565990086503
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
path = "../ShrinkMatrices"
npz_filename = path + "rating_matrix_shrunk.npz"

sparse_rating_matrix = sp.load_npz(npz_filename)

dense_rating_matrix = sparse_rating_matrix.todense()

## Train Test Split
Note that we used a seed in our random to ensure that all of our models are train test splitting the same way.

In [5]:
# X_tr now is 1d array version of dense_rating_matrix
X_tr = np.asarray(dense_rating_matrix.copy())
X_tr = X_tr.flatten()


nonzero_pairs = np.nonzero(X_tr)[0]
num_non_zero_pairs = len(nonzero_pairs)

total_num_pairs = X_tr.shape[0]
num_testing_pairs = int(0.1 * num_non_zero_pairs)


# seeds the random generator
np.random.seed(0)

# indices of 1d array X_tr
testing_pair_indices = np.random.choice(nonzero_pairs, num_testing_pairs, replace=False)
training_pair_indices = list(set(np.arange(total_num_pairs)) - set(testing_pair_indices))


X_te = X_tr.copy()

# sets testing pairs in training set to be 0
X_tr[testing_pair_indices] = 0

# sets training pairs in testing set to be 0
X_te[training_pair_indices] = 0


# takes X_tr and X_te back to shape of dense_rating_matrix

X_tr = X_tr.reshape((dense_rating_matrix.shape[0], dense_rating_matrix.shape[1]))
X_te = X_te.reshape((dense_rating_matrix.shape[0], dense_rating_matrix.shape[1]))


In [6]:
dense_rating_matrix = None
testing_pair_indices = None
training_pair_indices = None
gc.collect()

7

In [7]:
nonzero_users_tr, nonzero_books_tr, nonzero_ratings_tr = sp.find(X_tr)

In [8]:
X_tr = None
gc.collect()

0

In [9]:
nonzero_users_te, nonzero_books_te, nonzero_ratings_te = sp.find(X_te)

In [10]:
X_te = None
gc.collect()

0

In [11]:
P = None
sparse_rating_matrix = None
gc.collect()

0

## Build the Architecture of the Neural Network

In [54]:
k = 99

user_input = Input(shape=(1,), dtype='int32')
user_vector = Embedding(input_dim=max(nonzero_users_tr)+1,
                        output_dim=k, name="user_vector")(user_input)

book_input = Input(shape=(1,), dtype='int32')
book_vector = Embedding(input_dim=max(nonzero_books_tr)+1,
                        output_dim=k, name="book_vector")(book_input)

product = merge.multiply([user_vector, book_vector])
product = Dropout(0.2)(product)

layer = Flatten()(product)
layer = Dense(1, activation="relu")(layer)

output = Dense(1)(layer)

model = Model(inputs=[user_input, book_input], outputs=output)
model.compile(loss="mean_squared_error", optimizer='adam', metrics=["mean_squared_error", "mae"])

## Train the Model

In [55]:
class History(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.mse = []
        self.mae = []

    def on_epoch_end(self, batch, logs={}):
        self.mse.append(logs.get('mean_sqaured_error'))
        self.mae.append(logs.get('mae'))
        
history = History()

model.fit([np.array([[user] for user in nonzero_users_tr]),
           np.array([[book] for book in nonzero_books_tr])],
                    np.array([[rating] for rating in nonzero_ratings_tr]),
                    epochs=10, verbose=1, validation_split=0.2, callbacks=[history])

Train on 120751 samples, validate on 30188 samples
Epoch 1/10
120751/120751 [==============================] - 23s 193us/step - loss: 3.0862 - mean_squared_error: 3.0862 - mean_absolute_error: 1.2772 - val_loss: 1.2362 - val_mean_squared_error: 1.2362 - val_mean_absolute_error: 0.9626
Epoch 2/10
120751/120751 [==============================] - 23s 187us/step - loss: 0.5166 - mean_squared_error: 0.5166 - mean_absolute_error: 0.5373 - val_loss: 1.2827 - val_mean_squared_error: 1.2827 - val_mean_absolute_error: 0.9869
Epoch 3/10
120751/120751 [==============================] - 23s 187us/step - loss: 0.2602 - mean_squared_error: 0.2602 - mean_absolute_error: 0.3806 - val_loss: 1.2978 - val_mean_squared_error: 1.2978 - val_mean_absolute_error: 0.9933
Epoch 4/10
120751/120751 [==============================] - 23s 187us/step - loss: 0.1694 - mean_squared_error: 0.1694 - mean_absolute_error: 0.3158 - val_loss: 1.2449 - val_mean_squared_error: 1.2449 - val_mean_absolute_error: 0.9643
Epoch 5/1

In [ ]:
#Save the model
model.save("nn_model")

# Calculate MSE

In [56]:
# testing mse

predictions_te = model.predict([np.array([[user] for user in nonzero_users_te]),
                                np.array([[book] for book in nonzero_books_te])]).flatten()
np.mean((predictions_te - nonzero_ratings_te) ** 2)

0.7679616366976831

In [14]:
predictions_te

array([3.970225 , 3.8007812, 5.084221 , ..., 3.8157582, 3.7499943,
       3.8985112], dtype=float32)

In [57]:
# training mse

predictions_tr = model.predict([np.array([[user] for user in nonzero_users_tr]),
                                np.array([[book] for book in nonzero_books_tr])]).flatten()
np.sum((predictions_tr - nonzero_ratings_tr) ** 2) / len(nonzero_ratings_tr)

0.264326357370608